In [ ]:
import requests
import pyotp
import time

base_url = "http://localhost:12437"
failures = 0
test_email = "test@test.com"


def register_user():
    login_response = requests.post(
        f"{base_url}/v1/user",
        json={
            "email": test_email,
            "first_name": "Test",
            "last_name": "Name",
            "company_name": "Test Company",
            "job_title": "Test Job Title",
        },
    )
    response = login_response.json()
    print(response)
    if "otp_uri" in response:
        mfa_token = response["otp_uri"].split("secret=")[1].split("&")[0]
        totp = pyotp.TOTP(mfa_token)
        return f"Success! Your MFA token is: {totp.now()}"
    else:
        return response


while failures < 100:
    try:
        otp_uri = register_user()
        print(otp_uri)
        break
    except Exception as e:
        print(e)
        failures += 1
        time.sleep(5)

In [ ]:
import requests

response = requests.get(
    f"{base_url}/v1/user/exists",
    params={"email": test_email},
)
print(response.json())

In [ ]:
totp_token = otp_uri.split("secret=")[1].split("&")[0]
totp = pyotp.TOTP(totp_token)
print(totp.now())

response = requests.post(
    f"{base_url}/v1/login",
    json={"email": test_email, "token": totp.now()},
)
response = response.json()
if "detail" in response:
    detail = response["detail"]
    if str(detail).startswith("http"):
        token = detail.split("token=")[1]
        print(token)
    else:
        print(response)

In [ ]:
response = requests.put(
    f"{base_url}/v1/user",
    json={
        "first_name": "Test",
        "last_name": "Man",
        "company_name": "Test Company",
        "job_title": "Test Job Title",
    },
    headers={"Authorization": f"{token}"},
)
print(response.json())

In [ ]:
response = requests.get(
    f"{base_url}/v1/user",
    headers={"Authorization": f"{token}"},
)
print(response.json())